In [2]:
!pip install psycopg2

    100% |████████████████████████████████| 2.6MB 259kB/s eta 0:00:01


In [4]:
!pip install wikipedia

In [6]:
!pip install spacy

  Using cached plac-0.9.6-py2.py3-none-any.whl
  Using cached tqdm-4.15.0-py2.py3-none-any.whl


In [7]:
import request_category as rc
import database as db
from download import download
import search
import classify

#import pickle
from pathlib import Path
import re

import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelBinarizer, LabelEncoder

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
import sklearn.externals.joblib

In [ ]:
download( ['machine learning','business software'], depth = s) #pckle = False

In [9]:
db.query_to_dataframe("SELECT COUNT(*) FROM pages;")  #  LIMIT 1

,count
0,0


In [10]:
db.query_to_dataframe("SELECT COUNT(*) FROM categories;") # LIMIT 1

,count
0,0


In [11]:
db.query_to_dataframe("SELECT COUNT(*) FROM subcategories;") #  LIMIT 1

,count
0,0


In [12]:
db.query_to_dataframe("SELECT COUNT(*) FROM page_category;") # LIMIT 1

,count
0,0


In [ ]:
query_all = """SELECT crap.category, crap.subcategory, crap.pageid, p.title, p.article
            FROM (SELECT stuff.category, stuff.subcategory, pc.pageid 
                    FROM ( SELECT category, subcategory, subcategory_id 
                            FROM subcategories sc 
                            JOIN categories c 
                            ON sc.category_id = c.category_id ) as stuff 
                    JOIN page_category pc ON stuff.subcategory_id = pc.subcategory_id) as crap 
            INNER JOIN pages p 
            ON crap.pageid = p.pageid;"""

query_all = re.sub('\s+', ' ', query_all)
all_df = db.query_to_dataframe( query_all)
all_df.shape

In [ ]:
inner_query = """SELECT scc.category, scc.subcategory, scc.subcategory_id, pc.pageid  
    FROM page_category pc
    LEFT JOIN ( SELECT category, sc.subcategory, sc.subcategory_id 
                FROM subcategories sc
                LEFT JOIN categories c
                ON sc.category_id = c.category_id
                WHERE category = 'machine learning') scc
    ON pc.subcategory_id = scc.subcategory_id"""

query = """SELECT i.category, i.subcategory, COUNT( i.pageid) 
    FROM ({}) as i
    INNER JOIN pages p
    ON i.pageid = p.pageid
    GROUP BY i.category, i.subcategory""".format(inner_query)  #COUNT(*)

query = re.sub('\s+', ' ', query)

ml_df = db.query_to_dataframe( query)
ml_df.head(50)  ## Currently database includes articles at level 0 (machine learning),  pages from subcategories